In [2]:
import os

## Read file fuctions

In [121]:
def is_there_a_file(path: str) -> bool:
    return os.path.exists(path)

def check_file_extension(path: str, extension: str) -> bool:
    return os.path.splitext(path)[1] == extension

def valid_path_file(path: str)-> bool:
    return is_there_a_file(path) and check_file_extension(path, '.txt')

def get_all_files_by_extension(path: str, extension: str)-> list:
    return [file_name for file_name in os.listdir(path) if extension in file_name]

def read_file(path: str):
    all_data = []
    text = []
    with open(path, 'r') as file:
        for line in file:
            line = line.rstrip('\n').split(',')

            xn_and_yn = turn_all_elements_into_int(line[:8].copy())
            text_data = process_text_data(line[8:].copy())

            
            xn_and_yn.append(text_data)
            
            text.append(text_data)
            all_data.append(xn_and_yn)
        
        return all_data, text

def turn_all_elements_into_int(list_str: list)-> list:
    return list(map(lambda number: int(number), list_str))

def process_text_data(list_text: list)-> str:
    if len(list_text) > 1:
        list_text = ','.join(list_text)
    else:
        list_text = list_text[0]
        
    return list_text


In [ ]:
file_path = './test_files/OCR1.txt'
directory_path = './test_files'

valid_path_file(file_path)
get_all_files_by_extension(directory_path,'.txt')
all, text = read_file(file_path)
text

# Process fuctions

In [188]:
def find_indexes_equal_character(block: list, element: str)-> list:
    return [index for index, text_element in enumerate(block) if  element == text_element]

def find_indexes_that_contain_character(block: list, element: str)-> list:
    return [index for index, text_element in enumerate(block) if  element in text_element]

def index_before_2_and_after_2(index:int)->list:
    return [index - 2, index - 1, index + 1, index + 2]

def get_line_item(text: list, index_x: int)-> list:
    return [text[index] for index in index_before_2_and_after_2(index_x)]

def split_block_by_index(block: list, start: int = None, end: int = None)-> list:
    return block[start:end]

def get_main_blocks(text: list, index_invoice: int, index_item: int)-> int:
    indexes = [None, index_invoice, index_item, None]
    blocks = []

    for index in range(len(indexes) - 1):
        blocks.append(split_block_by_index(text,start=indexes[index], end=indexes[index + 1]))

    return blocks

def get_all_line_items(line_items_block: list)-> list:
    indexes_x = find_indexes_equal_character(line_items_block, 'X')
    result = []

    for index in indexes_x:
        result.append(get_line_item(line_items_block, index))

    return result


In [184]:
index_invoice = find_indexes_equal_character(text,'-INVOICE-')[0]
index_item = find_indexes_that_contain_character(text, 'ITEM')[0]

blocks = get_main_blocks(text, index_invoice, index_item)

In [190]:
index_x = find_indexes_equal_character(blocks[1], 'X')

get_line_item(blocks[1], index_x[0])

get_all_line_items(blocks[1])